# Private 10위(0.83173) Efficientnet + Ensemble

안녕하세요 팀 AGSC 입니다!
대회하느라 다들 고생하셨습니다.

먼저 Pytorch로 Efficientnet B1~B6 모델을 학습하여 앙상블 하였고,
다음 TensorFlow로 Efficientnet B1~B6 모델을 학습하여 앙상블하여
최종 적으로 Pytorch 모델과 TensorFlow 모델을 앙상블 하여 결과를 냈습니다.


1. Pytorch Model
    - 1. import Library
    - 2. Make PyTorch Dataset
    - 3. Make PyTorch Model
    - 4. Train PyTorch Model
    - 5. Test PyTorch Model
2. Tensorflow Model
    - 1. import Library
    - 2. Make TF Dataset
    - 3. Make TF Model
    - 4. Train TF Model
    - 5. Test TF Model
3. Ensemble PyTorch & TensorFlow
4. Make Submission

## Pytorch Model

### 1.Import Library

In [13]:
import random
import pandas as pd
import numpy as np
import os
import cv2

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 
import torch, gc

from torch.optim  import lr_scheduler

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [15]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':30,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [16]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### 2. Make PyTorch Dataset

In [17]:
os.getcwd()

'D:\\ML\\Artist Classification'

In [18]:
base_dir = 'D:/ML/Artist Classification'
data_dir = 'D:/ML/Artist Classification/open'


df = pd.read_csv(f'{data_dir}/train.csv')
df1 = pd.read_csv(f'{data_dir}/train.csv')

df.head()

,id,img_path,artist
0,0,./train/0000.jpg,Diego Velazquez
1,1,./train/0001.jpg,Vincent van Gogh
2,2,./train/0002.jpg,Claude Monet
3,3,./train/0003.jpg,Edgar Degas
4,4,./train/0004.jpg,Hieronymus Bosch


In [19]:
# Label Encoding
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)

In [20]:
df.head()

,id,img_path,artist
0,0,./train/0000.jpg,9
1,1,./train/0001.jpg,48
2,2,./train/0002.jpg,7
3,3,./train/0003.jpg,10
4,4,./train/0004.jpg,24


In [21]:
train_df, val_df, _, _ = train_test_split(df, df['artist'].values, test_size=0.2, random_state=CFG['SEED'])

In [22]:
train_df = train_df.sort_values(by=['id'])
train_df.head()

,id,img_path,artist
0,0,./train/0000.jpg,9
2,2,./train/0002.jpg,7
3,3,./train/0003.jpg,10
5,5,./train/0005.jpg,38
6,6,./train/0006.jpg,43


In [23]:
val_df = val_df.sort_values(by=['id'])
val_df.head()

,id,img_path,artist
1,1,./train/0001.jpg,48
4,4,./train/0004.jpg,24
17,17,./train/0017.jpg,10
21,21,./train/0021.jpg,29
29,29,./train/0029.jpg,28


In [24]:
def get_data(df, infer=False):
    if infer:
        return df['img_path'].values
    return df['img_path'].values, df['artist'].values

In [25]:
train_img_paths, train_labels = get_data(train_df)
val_img_paths, val_labels = get_data(val_df)

In [26]:
print(f"{data_dir}/{train_img_paths[0].split('/')[-2]}/{train_img_paths[0].split('/')[-1]}")

D:/ML/Artist Classification/open/train/0000.jpg


In [27]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        image = cv2.imread(f"{data_dir}/{img_path.split('/')[-2]}/{img_path.split('/')[-1]}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_paths)

In [28]:
def resize_transform(size, state='train'):
    if state == 'train':
        transform = A.Compose([
                                A.HorizontalFlip(p=0.5),
                                A.VerticalFlip(p=0.5),
                                A.Rotate(p=0.5),
                                A.RandomRotate90(p=0.5),
                                A.RandomResizedCrop(height=size, width=size, scale=(0.3, 1.0)),
                                A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                ToTensorV2(),
                                ])
    else:
        transform = A.Compose([
                            A.Resize(size,size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

    return transform
# 256, 288, 320, 384, 456, 528
# 240, 260, 300, 380, 456, 528

### 3.Make PyTorch Model

In [29]:
class BaseModel(nn.Module): # 224
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)

        self.classifier = nn.Sequential(nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(1000, num_classes))

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

class EfficientNet_B1(nn.Module): # 256
    def __init__(self, num_classes=len(le.classes_)):
        super(EfficientNet_B1, self).__init__()
        self.backbone = models.efficientnet_b1(pretrained=True)

        self.classifier = nn.Sequential(
                                        nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(1000, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

class EfficientNet_B2(nn.Module): # 288
    def __init__(self, num_classes=len(le.classes_)):
        super(EfficientNet_B2, self).__init__()
        self.backbone = models.efficientnet_b2(pretrained=True)

        self.classifier = nn.Sequential(
                                        nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(1000, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

class EfficientNet_B3(nn.Module): # 320
    def __init__(self, num_classes=len(le.classes_)):
        super(EfficientNet_B3, self).__init__()
        self.backbone = models.efficientnet_b3(pretrained=True)

        self.classifier = nn.Sequential(
                                        nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(1000, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x
    
class EfficientNet_B4(nn.Module): # 384
    def __init__(self, num_classes=len(le.classes_)):
        super(EfficientNet_B4, self).__init__()
        self.backbone = models.efficientnet_b4(pretrained=True)

        self.classifier = nn.Sequential(
                                        nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(1000, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

class EfficientNet_B5(nn.Module): # 456
    def __init__(self, num_classes=len(le.classes_)):
        super(EfficientNet_B5, self).__init__()
        self.backbone = models.efficientnet_b5(pretrained=True)

        self.classifier = nn.Sequential(
                                        nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(1000, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

class EfficientNet_B6(nn.Module): # 528
    def __init__(self, num_classes=len(le.classes_)):
        super(EfficientNet_B6, self).__init__()
        self.backbone = models.efficientnet_b6(pretrained=True)

        self.classifier = nn.Sequential(
                                        nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(1000, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x   

### 4.Train PyTorch Model

In [30]:
def train(model,epochs, optimizer, train_loader, test_loader, scheduler, device):
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1,epochs+1):
        model.train()
        train_loss = []
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.type(torch.LongTensor).to(device)
            
            optimizer.zero_grad()

            model_pred = model(img)
            
            loss = criterion(model_pred, label)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        tr_loss = np.mean(train_loss)
            
        val_loss, val_score = validation(model, criterion, test_loader, device)
            
        print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step()
            
        if best_score < val_score:
            best_model = model
            best_score = val_score
        
    return best_model

In [31]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):
    model.eval()
    
    model_preds = []
    true_labels = []
    
    val_loss = []
    
    with torch.no_grad():
        for img, label in tqdm(iter(test_loader)):
            img, label = img.float().to(device), label.type(torch.LongTensor).to(device)
            
            model_pred = model(img)
            
            loss = criterion(model_pred, label)
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
            
        
    val_f1 = competition_metric(true_labels, model_preds)

    return np.mean(val_loss), val_f1

In [32]:
num_epoch  = 30

In [33]:
train_dataset_0 = CustomDataset(train_img_paths, train_labels, resize_transform(224,'train'))
train_loader_0 = DataLoader(train_dataset_0, batch_size = 32, shuffle=True, num_workers=0)

val_dataset_0 = CustomDataset(val_img_paths, val_labels, resize_transform(224,'test'))
val_loader_0 = DataLoader(val_dataset_0, batch_size=32, shuffle=False, num_workers=0)

model_0 = BaseModel()
optimizer_0 = torch.optim.Adam(params = model_0.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_0 = lr_scheduler.MultiStepLR(optimizer_0, milestones=[10,20], gamma=0.5)
infer_model_0 = train(model_0,num_epoch, optimizer_0, train_loader_0, val_loader_0, scheduler_0, device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to C:\Users\DELL/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [76]:
torch.save(infer_model_0,'weights/b0_32_30.pt')
gc.collect()
torch.cuda.empty_cache()

In [77]:
#0.76917 batch:16, epoch:30, LR: [10,20]
#0.73733 batch:8, epoch:30, LR: [9,18,27]
train_dataset_1 = CustomDataset(train_img_paths, train_labels, resize_transform(256,'train'))
train_loader_1 = DataLoader(train_dataset_1, batch_size = 32, shuffle=True, num_workers=0)

val_dataset_1 = CustomDataset(val_img_paths, val_labels, resize_transform(256,'test'))
val_loader_1 = DataLoader(val_dataset_1, batch_size=32, shuffle=False, num_workers=0)

model_1 = EfficientNet_B1()
optimizer_1 = torch.optim.Adam(params = model_1.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_1 = lr_scheduler.MultiStepLR(optimizer_1, milestones=[10,20], gamma=0.5)
infer_model_1 = train(model_1, num_epoch, optimizer_1, train_loader_1, val_loader_1, scheduler_1, device)

  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

Epoch [1], Train Loss : [2.86433] Val Loss : [1.99244] Val F1 Score : [0.31057]


  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

...

  0%|          | 0/37 [00:00<?, ?it/s]

Epoch [29], Train Loss : [0.09446] Val Loss : [0.96864] Val F1 Score : [0.74452]


  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

Epoch [30], Train Loss : [0.09075] Val Loss : [1.01470] Val F1 Score : [0.71360]


In [78]:
torch.save(infer_model_1,'weights/b1_32_30.pt')
gc.collect()
torch.cuda.empty_cache()

In [79]:
#0.78464 batch:16, epoch:30, lr:[10,20]
#0.77673 batch:8, epoch:30, LR: [9,18,27]

train_dataset_2 = CustomDataset(train_img_paths, train_labels, resize_transform(288,'train'))
train_loader_2 = DataLoader(train_dataset_2, batch_size = 32, shuffle=True, num_workers=0)

val_dataset_2 = CustomDataset(val_img_paths, val_labels, resize_transform(288,'test'))
val_loader_2 = DataLoader(val_dataset_2, batch_size=32, shuffle=False, num_workers=0)

model_2 = EfficientNet_B2()
optimizer_2 = torch.optim.Adam(params = model_2.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_2 = lr_scheduler.MultiStepLR(optimizer_2, milestones=[10,20], gamma=0.5)
infer_model_2 = train(model_2, num_epoch, optimizer_2, train_loader_2, val_loader_2, scheduler_2, device)

  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

Epoch [1], Train Loss : [2.84091] Val Loss : [1.93657] Val F1 Score : [0.31513]


  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

...

  0%|          | 0/37 [00:00<?, ?it/s]

Epoch [29], Train Loss : [0.07385] Val Loss : [0.91839] Val F1 Score : [0.74788]


  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

Epoch [30], Train Loss : [0.06524] Val Loss : [0.91463] Val F1 Score : [0.75428]


In [80]:
torch.save(infer_model_2,'weights/b2_32_30.pt')
gc.collect()
torch.cuda.empty_cache()

In [81]:
# 0.77395 batch:32, epoch:30, LR: [10,20]
# 0.77227 batch:16, epcoh:30, lr:[10,20]
# 0.76952 batch:8, epoch:30, LR: [9,18,27]

train_dataset_3 = CustomDataset(train_img_paths, train_labels, resize_transform(320,'train'))
train_loader_3 = DataLoader(train_dataset_3, batch_size = 32, shuffle=True, num_workers=0)

val_dataset_3 = CustomDataset(val_img_paths, val_labels, resize_transform(320,'test'))
val_loader_3 = DataLoader(val_dataset_3, batch_size=32, shuffle=False, num_workers=0)

model_3 = EfficientNet_B3()
optimizer_3 = torch.optim.Adam(params = model_3.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_3 = lr_scheduler.MultiStepLR(optimizer_3, milestones=[10,20], gamma=0.5)
infer_model_3 = train(model_3, num_epoch, optimizer_3, train_loader_3, val_loader_3, scheduler_3, device)

  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

Epoch [1], Train Loss : [2.96602] Val Loss : [2.09809] Val F1 Score : [0.25765]


  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

...

  0%|          | 0/37 [00:00<?, ?it/s]

Epoch [29], Train Loss : [0.07765] Val Loss : [0.81785] Val F1 Score : [0.76842]


  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

Epoch [30], Train Loss : [0.07697] Val Loss : [1.13450] Val F1 Score : [0.76876]


In [82]:
torch.save(infer_model_3,'weights/b3_32_30.pt')
gc.collect()
torch.cuda.empty_cache()

In [83]:
#0.79137 batch:16, epoch:30, lr[10,20]
#0.79599 batch:8, epoch:30, LR: [9,18,27]

train_dataset_4 = CustomDataset(train_img_paths, train_labels, resize_transform(384,'train'))
train_loader_4 = DataLoader(train_dataset_4, batch_size = 16, shuffle=True, num_workers=0)

val_dataset_4 = CustomDataset(val_img_paths, val_labels, resize_transform(384,'test'))
val_loader_4 = DataLoader(val_dataset_4, batch_size=16, shuffle=False, num_workers=0)

model_4 = EfficientNet_B4()
optimizer_4 = torch.optim.Adam(params = model_4.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_4 = lr_scheduler.MultiStepLR(optimizer_4, milestones=[10,20], gamma=0.5)
infer_model_4 = train(model_4, num_epoch, optimizer_4, train_loader_4, val_loader_4, scheduler_4, device)

  0%|          | 0/296 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

Epoch [1], Train Loss : [2.91452] Val Loss : [2.01377] Val F1 Score : [0.29155]


  0%|          | 0/296 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

...

  0%|          | 0/74 [00:00<?, ?it/s]

Epoch [29], Train Loss : [0.12939] Val Loss : [1.39810] Val F1 Score : [0.75337]


  0%|          | 0/296 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

Epoch [30], Train Loss : [0.12304] Val Loss : [0.89081] Val F1 Score : [0.77000]


In [85]:
torch.save(infer_model_4,'weights/b4_16_30.pt')
gc.collect()
torch.cuda.empty_cache()

In [86]:
#0.82004 batch:8, epoch : 30, lr:[9,18,27]
train_dataset_5 = CustomDataset(train_img_paths, train_labels, resize_transform(456,'train'))
train_loader_5 = DataLoader(train_dataset_5, batch_size = 8, shuffle=True, num_workers=0)

val_dataset_5 = CustomDataset(val_img_paths, val_labels, resize_transform(456,'test'))
val_loader_5 = DataLoader(val_dataset_5, batch_size = 8, shuffle=False, num_workers=0)

model_5 = EfficientNet_B5()
optimizer_5 = torch.optim.Adam(params = model_5.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_5 = lr_scheduler.MultiStepLR(optimizer_5, milestones=[10,20], gamma=0.5)
infer_model_5 = train(model_5, num_epoch, optimizer_5, train_loader_5, val_loader_5, scheduler_5, device)

  0%|          | 0/591 [00:00<?, ?it/s]

  0%|          | 0/148 [00:00<?, ?it/s]

Epoch [1], Train Loss : [2.51193] Val Loss : [1.97491] Val F1 Score : [0.30960]


  0%|          | 0/591 [00:00<?, ?it/s]

  0%|          | 0/148 [00:00<?, ?it/s]

...

  0%|          | 0/148 [00:00<?, ?it/s]

Epoch [29], Train Loss : [0.11044] Val Loss : [0.87055] Val F1 Score : [0.79833]


  0%|          | 0/591 [00:00<?, ?it/s]

  0%|          | 0/148 [00:00<?, ?it/s]

Epoch [30], Train Loss : [0.09429] Val Loss : [0.84395] Val F1 Score : [0.79083]


In [87]:
torch.save(infer_model_5,'weights/b5_8_30.pt')
gc.collect()
torch.cuda.empty_cache()

In [88]:
#0.80073 batch:4, epoch:30, lr:[8,16,24]
train_dataset_6 = CustomDataset(train_img_paths, train_labels, resize_transform(528,'train'))
train_loader_6 = DataLoader(train_dataset_6, batch_size = 4, shuffle=True, num_workers=0)

val_dataset_6 = CustomDataset(val_img_paths, val_labels, resize_transform(528,'test'))
val_loader_6 = DataLoader(val_dataset_6, batch_size=4, shuffle=False, num_workers=0)

model_6 = EfficientNet_B6()
optimizer_6 = torch.optim.Adam(params = model_6.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = lr_scheduler.MultiStepLR(optimizer_6, milestones=[10,20], gamma=0.5)
infer_model_6 = train(model_6,num_epoch, optimizer_6, train_loader_6, val_loader_6, scheduler, device)

  0%|          | 0/1182 [00:00<?, ?it/s]

  0%|          | 0/296 [00:00<?, ?it/s]

Epoch [1], Train Loss : [2.75712] Val Loss : [2.27502] Val F1 Score : [0.26730]


  0%|          | 0/1182 [00:00<?, ?it/s]

  0%|          | 0/296 [00:00<?, ?it/s]

...

  0%|          | 0/296 [00:00<?, ?it/s]

Epoch [29], Train Loss : [0.18628] Val Loss : [1.16049] Val F1 Score : [0.76815]


  0%|          | 0/1182 [00:00<?, ?it/s]

  0%|          | 0/296 [00:00<?, ?it/s]

Epoch [30], Train Loss : [0.17886] Val Loss : [1.06788] Val F1 Score : [0.77537]


In [89]:
torch.save(infer_model_6,'weights/b6_4_30.pt')
gc.collect()
torch.cuda.empty_cache()

### 5. Test PyTorch Model

In [35]:
test_df = pd.read_csv(f'{data_dir}/test.csv')
test_df.head()


,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg


In [36]:
test_img_paths = get_data(test_df, infer=True)

In [37]:
test_dataset_1 = CustomDataset(test_img_paths, None, resize_transform(256,'test'))
test_loader_1 = DataLoader(test_dataset_1, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

test_dataset_2 = CustomDataset(test_img_paths, None, resize_transform(288,'test'))
test_loader_2 = DataLoader(test_dataset_2, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
                               
test_dataset_3 = CustomDataset(test_img_paths, None, resize_transform(320,'test'))
test_loader_3 = DataLoader(test_dataset_3, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
                               
test_dataset_4 = CustomDataset(test_img_paths, None, resize_transform(384,'test'))
test_loader_4 = DataLoader(test_dataset_4, batch_size=16, shuffle=False, num_workers=0)
                               
test_dataset_5 = CustomDataset(test_img_paths, None, resize_transform(456,'test'))
test_loader_5 = DataLoader(test_dataset_5, batch_size=16, shuffle=False, num_workers=0)

test_dataset_6 = CustomDataset(test_img_paths, None, resize_transform(528,'test'))
test_loader_6 = DataLoader(test_dataset_6, batch_size=8, shuffle=False, num_workers=0)

                               

# Inference Test

In [23]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)
            
            model_pred = model(img)
            model_preds += model_pred.detach().cpu().numpy().tolist()
            
    
    print('Done.')
    return model_preds

In [24]:
#preds = inference(infer_model_5, test_loader_5, device)

## TF model

### 1.Import Library

In [38]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os
import math
import scipy as sp
import PIL
import logging
import warnings
import matplotlib.style as style
import seaborn as sns
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
# Tensorflow
from tensorflow.keras import models, layers, Model, regularizers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D 
from sklearn import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau,  EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB6, ResNet50V2, EfficientNetB0
#from keras_tuner.tuners import RandomSearch
from tensorflow.keras.utils import Sequence
#import scikitplot as skplt
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
#from albumentations import Compose, HorizontalFlip, CLAHE, HueSaturationValue,RandomBrightness, RandomContrast, RandomGamma,ToFloat, ShiftScaleRotate
import albumentations as A


### 2.Make Dataset

In [39]:
class CustomTFDataset(Sequence):
    def __init__(self, x_set, y_set, batch_size, augmentations, shuffle = True):
        self.y = y_set
        self.x = x_set
        self.batch_size = batch_size
        self.augment = augmentations
        self.indexes = np.arange(self.x.shape[0])
        self.shuffle = shuffle
        self.on_epoch_end()
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    # 지정 배치 크기만큼 데이터를 로드합니다.
    def __getitem__(self, idx):
        inds = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
        img_path = self.x[inds]
        batch_x = [cv2.cvtColor(cv2.imread(f"{data_dir}/{x.split('/')[-2]}/{x.split('/')[-1]}"), cv2.COLOR_BGR2RGB)for x in img_path]
        #batch_x = self.x[inds]
        batch_y = self.y[inds]
        
        # augmentation을 적용해서 numpy array에 stack합니다.
        return np.stack([
            self.augment(image=x)["image"] for x in batch_x
        ], axis=0), np.array(batch_y)

In [40]:
input_size = [224, 240, 260, 300 ,380, 456, 528, 600]

In [41]:
train_transform = [A.Compose([A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5), A.Rotate(limit= 90),
                              #A.ToGray(p=0.2), A.RandomBrightnessContrast(0.2, 0.2, p=0.2),
                              A.RandomResizedCrop(X, X, (0.08, 1.0)),
                              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                   ])
                   for X in input_size]

test_transform = [A.Compose([A.Resize(X,X),A.Normalize(mean=(0.485, 0.456, 0.406),
                            std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0)])
                  for X in input_size]

In [42]:
one_train_labels = tf.keras.utils.to_categorical(train_labels)
one_val_labels = tf.keras.utils.to_categorical(val_labels)

In [43]:
train_gen = [CustomTFDataset(train_img_paths, one_train_labels, 16 if x < 5 else 4 , TT, shuffle =True)
            for x ,TT in enumerate (train_transform)]
val_gen = [CustomTFDataset(val_img_paths, one_val_labels, 16 if x < 5 else 4, TT, shuffle =False)
          for x, TT in enumerate (test_transform)]

### 3.Make TF Model

In [44]:
ensemble_model = []

In [45]:
def ada(e): 
    return tf.keras.optimizers.Adam(learning_rate=e)

In [46]:
#0
base_model = tf.keras.applications.EfficientNetB0(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=1000,
                   weights='imagenet')
X = Flatten()(base_model.output)
#X = Dense(1280, activation = 'relu')(X)
#X = BatchNormalization()(X)
#X = Dropout(0.2)(X)
X = Dense(500, activation = 'relu')(X)
X = BatchNormalization()(X)
X = Dropout(0.2)(X)
X = Dense(50, activation='softmax')(base_model.output)
image_model = Model(inputs = base_model.input, outputs = X)
image_model.compile(optimizer=ada(3e-4), loss='categorical_crossentropy', metrics=['accuracy'])
ensemble_model.append(image_model)

In [47]:
#1
base_model = tf.keras.applications.EfficientNetB1(include_top=False,
                   input_shape=(240,240,3),
                   pooling='avg',classes=1000,
                   weights='imagenet')
X = Flatten()(base_model.output)
#X = Dense(1280, activation = 'relu')(X)
#X = BatchNormalization()(X)
#X = Dropout(0.2)(X)
X = Dense(500, activation = 'relu')(X)
X = BatchNormalization()(X)
X = Dropout(0.2)(X)
X = Dense(50, activation='softmax')(X)
image_model = Model(inputs = base_model.input, outputs = X)
image_model.compile(optimizer=ada(3e-4), loss='categorical_crossentropy', metrics=['accuracy'])
ensemble_model.append(image_model)

In [48]:
#2
base_model = tf.keras.applications.EfficientNetB2(include_top=False,
                   input_shape=(260,260,3),
                   pooling='avg',classes=1000,
                   weights='imagenet')
X = Flatten()(base_model.output)
#X = Dense(1408, activation = 'relu')(X)
#X = BatchNormalization()(X)
#X = Dropout(0.2)(X)
X = Dense(500, activation = 'relu')(X)
X = BatchNormalization()(X)
X = Dropout(0.2)(X)
X = Dense(50, activation='softmax')(X)
image_model = Model(inputs = base_model.input, outputs = X)
image_model.compile(optimizer=ada(3e-4), loss='categorical_crossentropy', metrics=['accuracy'])
ensemble_model.append(image_model)

In [49]:
#3
base_model = tf.keras.applications.EfficientNetB3(include_top=False,
                   input_shape=(300,300,3),
                   pooling='avg',classes=1000,
                   weights='imagenet')
X = Flatten()(base_model.output)
#X = Dense(1536, activation = 'relu')(X)
#X = BatchNormalization()(X)
#X = Dropout(0.2)(X)
X = Dense(500, activation = 'relu')(X)
X = BatchNormalization()(X)
X = Dropout(0.2)(X)
X = Dense(50, activation='softmax')(X)
image_model = Model(inputs = base_model.input, outputs = X)
image_model.compile(optimizer=ada(3e-4), loss='categorical_crossentropy', metrics=['accuracy'])
ensemble_model.append(image_model)

In [50]:
#4
base_model = tf.keras.applications.EfficientNetB4(include_top=False,
                   input_shape=(380,380,3),
                   pooling='avg',classes=1000,
                   weights='imagenet')
X = Flatten()(base_model.output)
#X = Dense(1792, activation = 'relu')(X)
#X = BatchNormalization()(X)
#X = Dropout(0.2)(X)
X = Dense(500, activation = 'relu')(X)
X = BatchNormalization()(X)
X = Dropout(0.2)(X)
X = Dense(50, activation='softmax')(X)
image_model = Model(inputs = base_model.input, outputs = X)
image_model.compile(optimizer=ada(3e-4), loss='categorical_crossentropy', metrics=['accuracy'])
ensemble_model.append(image_model)

In [51]:
#5
base_model = tf.keras.applications.EfficientNetB5(include_top=False,
                   input_shape=(456,456,3),
                   pooling='avg',classes=1000,
                   weights='imagenet')
X = Flatten()(base_model.output)
#X = Dense(2048, activation = 'relu')(X)
#X = BatchNormalization()(X)
#X = Dropout(0.2)(X)
X = Dense(500, activation = 'relu')(X)
X = BatchNormalization()(X)
X = Dropout(0.2)(X)
X = Dense(50, activation='softmax')(X)
image_model = Model(inputs = base_model.input, outputs = X)
image_model.compile(optimizer=ada(3e-4), loss='categorical_crossentropy', metrics=['accuracy'])
ensemble_model.append(image_model)

In [52]:
#6
base_model = tf.keras.applications.EfficientNetB6(include_top=False,
                   input_shape=(528,528,3),
                   pooling='avg',classes=1000,
                   weights='imagenet')
X = Flatten()(base_model.output)
#X = Dense(2304, activation = 'relu')(X)
#X = BatchNormalization()(X)
#X = Dropout(0.2)(X)
X = Dense(500, activation = 'relu')(X)
X = BatchNormalization()(X)
X = Dropout(0.2)(X)
X = Dense(50, activation='softmax')(X)
image_model = Model(inputs = base_model.input, outputs = X)
image_model.compile(optimizer=ada(3e-4), loss='categorical_crossentropy', metrics=['accuracy'])
ensemble_model.append(image_model)

### 4.Train TF model

In [53]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15786879934872305681
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22720544768
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14073362446480609761
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [54]:
def ada(e): 
    return tf.keras.optimizers.Adam(learning_rate=e)

In [65]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
def mcp_save(name):
     return ModelCheckpoint('weights/'+name +'model.h5', save_best_only=True, monitor='val_accuracy', mode='max',save_weights_only=True)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, epsilon=1e-4, mode='min')

In [66]:
history= [train_model.fit(train_gen[name], epochs=30, validation_data=val_gen[name],
                          callbacks=[earlyStopping, mcp_save(str(name)), reduce_lr_loss])for name, train_model in enumerate (ensemble_model)]

Epoch 1/30
296/296 [==============================] - 61s 205ms/step - loss: 0.8231 - accuracy: 0.7570 - val_loss: 5.2849 - val_accuracy: 0.1378 - lr: 3.0000e-04
Epoch 2/30
 24/296 [=>............................] - ETA: 46s - loss: 0.8307 - accuracy: 0.7526

KeyboardInterrupt: 

In [ ]:
scores = [ensemble_model[x].evaluate_generator(val_gen[x], verbose=0) for x in range(7)]

In [ ]:
for x in range(7):
    print("%s: %.2f%%" %(ensemble_model[x].metrics_names[1], scores[x][1]*100))

In [ ]:
fig1 = plt.gcf()
for i in range(7):
    plt.plot(history[i].history['loss'])
    plt.plot(history[i].history['val_loss'])
    plt.axis(ymin=0.1,ymax=10)
    plt.axis(xmax = 30)
    plt.grid()
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend(['train', 'validation'])
plt.show()

### 5.Test TF model

In [39]:
for x in range(7):
    print(ensemble_model[x])
    ensemble_model[x].load_weights(filepath='weights/'+str(x)+'model.h5')

In [40]:
test_transform = [A.Compose([A.Resize(X,X),A.Normalize(mean=(0.485, 0.456, 0.406),
                            std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0)])
                  for X in input_size]

In [41]:
test_img_paths = get_data(test_df, infer=True)
test_gen = [CustomTFDataset(test_img_paths, test_img_paths, 16, x, shuffle =False)for x in test_transform]

In [42]:
test_gen[6].batch_size = 8

In [49]:
gc.collect()
torch.cuda.empty_cache()

In [44]:
Test_Predic = [ensemble_model[x].predict(test_gen[x])for x in range(1,7)]

1584/1584 [==============================] - 170s 105ms/step


## Ensemble PyTorch and TensorFlow

In [30]:
torch_model_1 = torch.load('weights/b1_32_30.pt')
torch_model_2 = torch.load('weights/b2_32_30.pt')
torch_model_3 = torch.load('weights/b3_32_30.pt')
torch_model_4 = torch.load('weights/b4_16_30.pt')
torch_model_5 = torch.load('weights/b5_8_30.pt')
torch_model_6 = torch.load('weights/b6_4_30.pt')

In [31]:
model_list = [torch_model_1,torch_model_2, torch_model_3,torch_model_4,torch_model_5,torch_model_6]
test_dataset_list = [ test_loader_1,test_loader_2, test_loader_3, test_loader_4, test_loader_5, test_loader_6]

pred = []
for i in range(len(model_list)):
    pred += [inference(model_list[i],test_dataset_list[i],device)]


  0%|          | 0/396 [00:00<?, ?it/s]

Done.


  0%|          | 0/396 [00:00<?, ?it/s]

Done.


  0%|          | 0/396 [00:00<?, ?it/s]

...

Done.


  0%|          | 0/792 [00:00<?, ?it/s]

Done.


  0%|          | 0/1584 [00:00<?, ?it/s]

Done.


In [45]:
pred_tf = Test_Predic

result = []
for i_0,i_1,i_2,i_3,i_4,i_5,i_tf0,i_tf1,i_tf2,i_tf3,i_tf4,i_tf5 in zip(pred[0],pred[1],pred[2],pred[3],pred[4],pred[5],
                                        pred_tf[0],pred_tf[1],pred_tf[2],pred_tf[3],pred_tf[4],pred_tf[5]):#,pred_tf[6]):
    i_0 = F.softmax(torch.Tensor(i_0))
    i_1 = F.softmax(torch.Tensor(i_1))
    i_2 = F.softmax(torch.Tensor(i_2))
    i_3 = F.softmax(torch.Tensor(i_3))
    i_4 = F.softmax(torch.Tensor(i_4))
    i_5 = F.softmax(torch.Tensor(i_5))

    i_tf0 = torch.Tensor(i_tf0)
    i_tf1 = torch.Tensor(i_tf1)
    i_tf2 = torch.Tensor(i_tf2)
    i_tf3 = torch.Tensor(i_tf3)
    i_tf4 = torch.Tensor(i_tf4)
    i_tf5 = torch.Tensor(i_tf5)
    #i_tf6 = torch.Tensor(i_tf6)

    i = i_0 + i_1 + i_2 + i_3 + i_4 +i_5 +i_tf0 +i_tf1 +i_tf2 +i_tf3 +i_tf4 +i_tf5 #+i_tf6

    i_arg = i.argmax()
    result.append(i_arg.item())
print(result)

[10, 2, 29, 0, 10, 33, 33, 48, 11, 40, 32, 43, 48, 10, 45, 33, 16, 33, 18, 38, 38, 22, 38, 15, 33, 38, 48, 48, 44, 33, 16, 32, 9, 21, 48, 10, 35, 4, 21, 46, 18, 45, 10, 10, 44, 3, 9, 42, 36, 10, 5, 47, 13, 10, 3, 2, 41, 13, 33, 29, 34, 35, 2, 15, 42, 10, 43, 42, 10, 46, 10, 0, 45, 32, 38, 15, 10, 30, 0, 48, 36, 24, 10, 7, 48, 33, 46, 40, 23, 43, 27, 1, 5, 27, 40, 5, 33, 20, 38, 1, 35, 22, 21, 18, 45, 46, 1, 48, 1, 0, 38, 38, 11, 21, 26, 48, 8, 33, 46, 17, 4, 2, 3, 42, 43, 37, 29, 48, 9, 42, 1, 43, 39, 40, 9, 29, 33, 1, 32, 46, 38, 9, 10, 29, 1, 10, 47, 37, 48, 20, 1, 37, 10, 10, 43, 46, 29, 16, 38, 41, 3, 15, 48, 38, 33, 34, 0, 38, 21, 48, 48, 11, 30, 23, 44, 35, 20, 0, 0, 42, 38, 38, 39, 42, 15, 10, 10, 0, 33, 10, 8, 35, 21, 15, 18, 30, 40, 40, 1, 10, 30, 0, 30, 15, 1, 10, 1, 21, 4, 38, 0, 48, 10, 0, 35, 19, 6, 24, 10, 10, 10, 2, 15, 48, 11, 10, 38, 1, 30, 33, 47, 32, 16, 24, 38, 35, 19, 4, 2, 21, 6, 35, 10, 7, 15, 2, 10, 39, 8, 4, 15, 0, 0, 45, 33, 24, 48, 45, 48, 46, 44, 30, 19, 19,

# Make Submission

In [46]:
preds = le.inverse_transform(result) # LabelEncoder로 변환 된 Label을 다시 화가이름으로 변환

In [47]:
print(le.classes_)

['Albrecht Du rer' 'Alfred Sisley' 'Amedeo Modigliani' 'Andrei Rublev'
 'Andy Warhol' 'Camille Pissarro' 'Caravaggio' 'Claude Monet'
 'Diego Rivera' 'Diego Velazquez' 'Edgar Degas' 'Edouard Manet'
 'Edvard Munch' 'El Greco' 'Eugene Delacroix' 'Francisco Goya'
 'Frida Kahlo' 'Georges Seurat' 'Giotto di Bondone' 'Gustav Klimt'
 'Gustave Courbet' 'Henri Matisse' 'Henri Rousseau'
 'Henri de Toulouse-Lautrec' 'Hieronymus Bosch' 'Jackson Pollock'
 'Jan van Eyck' 'Joan Miro' 'Kazimir Malevich' 'Leonardo da Vinci'
 'Marc Chagall' 'Michelangelo' 'Mikhail Vrubel' 'Pablo Picasso'
 'Paul Cezanne' 'Paul Gauguin' 'Paul Klee' 'Peter Paul Rubens'
 'Pierre-Auguste Renoir' 'Piet Mondrian' 'Pieter Bruegel' 'Raphael'
 'Rembrandt' 'Rene Magritte' 'Salvador Dali' 'Sandro Botticelli' 'Titian'
 'Vasiliy Kandinskiy' 'Vincent van Gogh' 'William Turner']


In [49]:
submit = pd.read_csv('./submit.csv')

In [50]:
submit['artist'] = preds
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Leonardo da Vinci
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Edgar Degas


In [51]:
submit.to_csv('submit2022-11-11.csv', index=False)